In [1]:
import os.path as op
import numpy as np
import pandas as pd
from mne.io import read_epochs_eeglab
import scipy.io
from superlet import superlet, scale_from_period
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [7]:
pipeline='NEARICA_behav'
age='9m'
c3_cluster_chans=['E16', 'E20', 'E21', 'E22']
c4_cluster_chans=['E41', 'E49', 'E50', 'E51']

In [5]:
subjects=pd.read_csv(op.join('/home/bonaiuto/dev_beta_umd/data',age,'data/participants.tsv'), sep='\t')

subject_ids = subjects['participant_id']

max_freq=80
num=160
foi = np.linspace(1, max_freq, num)
scales = scale_from_period(1/foi)

for subj_id in subject_ids:
    
    print(subj_id)

    subject_data_dir=op.join('/home/bonaiuto/dev_beta_umd/data',age, 'derivatives', pipeline, subj_id, 'processed_data')

    fname='{}_task-tool_obs_exe_eeg_processed_data.set'.format(subj_id)

    EEG=read_epochs_eeglab(op.join(subject_data_dir,fname))
    EEG_data=EEG.get_data()

    for cluster_name, channels in zip(['C3','C4'], [c3_cluster_chans, c4_cluster_chans]):

        output_array=np.zeros((EEG.get_data().shape[0],len(channels),len(foi),EEG.get_data().shape[2]))                
        for ix in range(EEG_data.shape[0]):
            epoch=EEG_data[ix,:,:]

            def do_superlet(signal, scales, sfreq):
                spec = superlet(
                    signal,
                    samplerate=sfreq,
                    scales=scales,
                    order_max=40,
                    order_min=1,
                    c_1=4,
                    adaptive=True,
                )
                return np.single(np.abs(spec))
            
            chan_idx = np.array([np.where(np.in1d(EEG.info.ch_names, ch))[0][0] for ch in channels])

            chan_tfs=Parallel(n_jobs=-1)(delayed(do_superlet)(epoch[c_ix,:], scales, EEG.info["sfreq"]) for c_ix in chan_idx)
            output_array[ix,:,:,:]=np.array(chan_tfs)

        fname = "processed_superlet_tf_{}.mat".format(cluster_name)
        output_path = op.join(subject_data_dir,fname)
        scipy.io.savemat(output_path,
                         {
                             'foi': foi,
                             'trial_tf': output_array
                         })            
        del output_array


sub-001
Extracting parameters from /home/bonaiuto/dev_beta_umd/data/adult/derivatives/NEARICA_behav/sub-001/processed_data/sub-001_task-tool_obs_exe_eeg_processed_data.set...
Not setting metadata
194 matching events found
No baseline correction applied
0 projection items activated
Ready.
sub-002
Extracting parameters from /home/bonaiuto/dev_beta_umd/data/adult/derivatives/NEARICA_behav/sub-002/processed_data/sub-002_task-tool_obs_exe_eeg_processed_data.set...
Not setting metadata
194 matching events found
No baseline correction applied
0 projection items activated
Ready.
sub-003
Extracting parameters from /home/bonaiuto/dev_beta_umd/data/adult/derivatives/NEARICA_behav/sub-003/processed_data/sub-003_task-tool_obs_exe_eeg_processed_data.set...
Not setting metadata
194 matching events found
No baseline correction applied
0 projection items activated
Ready.
sub-004
Extracting parameters from /home/bonaiuto/dev_beta_umd/data/adult/derivatives/NEARICA_behav/sub-004/processed_data/sub-004_ta